# ETL (Extract, Transform, Load) -- Ciudad Autonoma de Buenos Aires

## Siniestros viales

Nuestro objetivo general consiste en brindar a las autoridades, organizaciones de tráfico y ciudadanos en general una visión precisa y detallada de las zonas con mayor riesgo, los momentos críticos y los factores más influyentes en la seguridad vial. A través de esta información, buscamos fomentar la implementación de medidas preventivas y estrategias efectivas que contribuyan de manera significativa a la reducción de tragedias viales en la Ciudad Autónoma de Buenos Aires.

### IMPORTAMOS LIBRERÍAS NECESARIAS

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from functions import carga_datos_excel
from functions import analizar_valores_sd
from functions import extraer_coordenadas
import re
import sketch
import geopandas as gpd
import geojson
import folium
from folium.plugins import HeatMap
from summarytools import dfSummary

## RUTA DE LOS DATASET

In [24]:
# Llamamos a la funcion carga_datos_excel

homicidio =  carga_datos_excel('homicidios.xlsx', ['HECHOS', 'VICTIMAS'])
lesiones = carga_datos_excel('lesiones.xlsx', ['HECHOS', 'VICTIMAS'])
comunas= pd.read_excel("comunas.xlsx")

## ETL del DATASET de __HOMICIDIOS - HECHOS__

### EXPLORACION Y LIMPIEZA DE LOS DATOS

In [6]:
# Accedemos al dataframe por nombre de hoja, homicidios - hechos

df_hechos = homicidio['HECHOS']

In [7]:
#Visualizamos la primera fila del DataFrame.

print('DataFrame - Original: "HOMICIDIO" - "HECHOS"' )
df_hechos.head(5)

DataFrame - Original: "HOMICIDIO" - "HECHOS"


,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


In [8]:
#Obtener informacion general del Dataframe

df_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

In [9]:
#Exploracion de datos nulos y duplicados

df_hechos.isnull().sum()

ID                         0
N_VICTIMAS                 0
FECHA                      0
AAAA                       0
MM                         0
DD                         0
HORA                       0
HH                         0
LUGAR_DEL_HECHO            0
TIPO_DE_CALLE              0
Calle                      1
Altura                   567
Cruce                    171
Dirección Normalizada      8
COMUNA                     0
XY (CABA)                  0
pos x                      0
pos y                      0
PARTICIPANTES              0
VICTIMA                    0
ACUSADO                    0
dtype: int64

### Se analizan las columnas con datos nulos

In [10]:
df_hechos[["Altura", "Calle", "Dirección Normalizada", "Cruce"]][:5]

,Altura,Calle,Dirección Normalizada,Cruce
0,NaN,PIEDRA BUENA AV.,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...","FERNANDEZ DE LA CRUZ, F., GRAL. AV."
1,NaN,"PAZ, GRAL. AV.","PAZ, GRAL. AV. y DE LOS CORRALES AV.",DE LOS CORRALES AV.
2,2034.0,ENTRE RIOS AV.,ENTRE RIOS AV. 2034,NaN
3,NaN,LARRAZABAL AV.,"LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.","VILLEGAS, CONRADO, GRAL."
4,NaN,SAN JUAN AV.,"SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.","SAENZ PE?A, LUIS, PRES."


In [11]:
df_hechos[df_hechos["Dirección Normalizada"].isnull()][["Dirección Normalizada", "Cruce","Altura", "Calle"]]

,Dirección Normalizada,Cruce,Altura,Calle
38,NaN,NaN,NaN,"LUGONES, LEOPOLDO AV."
106,NaN,NaN,NaN,AUTOPISTA BUENOS AIRES - LA PLATA
119,NaN,NaN,NaN,NaN
180,NaN,NaN,NaN,AUTOPISTA PERITO MORENO
181,NaN,NaN,NaN,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.
313,NaN,NaN,NaN,"LUGONES, LEOPOLDO AV."
546,NaN,NaN,NaN,"LUGONES, LEOPOLDO AV."
621,NaN,NaN,NaN,AUTOPISTA BUENOS AIRES - LA PLATA


Cantidad de valores SD

In [12]:
# llamamos a la funcion analizar_valores_sd

resultados_sd = analizar_valores_sd(df_hechos)
resultados_sd

,Columna,Cantidad de SD,Porcentaje de SD
6,HORA,1,0.143678
7,HH,1,0.143678
8,LUGAR_DEL_HECHO,1,0.143678
19,VICTIMA,9,1.293103
20,ACUSADO,23,3.304598


Tras examinar datos nulos, que representan el 5,1% del dataset; distribuidos en tres columnas __Altura__, __Dirección Normalizada__, __Cruce__ y __Calle.__

    - Para el caso de __Calle__ tiene un solo dato nulo.
    - Para el caso de __Dirección Normalizada__ 8 nulos.
    - Para el caso de __Cruce__ tiene 171 nulos, siendo el 24,56% del los datos de la columna.
    - Para el caso de __Altura__ tiene 567 nulos que representan el 81,5% de datos de la columna

Analizando los datos de la Altura se procede a eliminarla debido a la gran cantidad de faltantes y no existiendo manera de poder rellenar las colummas; ya que altura se refiere a la numeración de la calle donde se produce el siniestro y la mayoria de los casos se producen en cruces o esquinas. La columna Cruce solo va a tener dato cuando el hecho se porduce en un cruce de calles, por lo que se conserva de esta manera. La columna Dirección Normalizada no tiene posibilidad de completarse con las columnas del dataset; pero contiene solo 1,1% de nulos por lo se conserva de esta manera.

In [13]:
df_hechos = df_hechos.drop("Altura", axis=1)
df_hechos.columns

Index(['ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH',
       'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'Calle', 'Cruce',
       'Dirección Normalizada', 'COMUNA', 'XY (CABA)', 'pos x', 'pos y',
       'PARTICIPANTES', 'VICTIMA', 'ACUSADO'],
      dtype='object')

In [14]:
df_hechos.duplicated().sum()

0

No se observan datos duplicados

### TRANFORMACIÓN DE LOS DATOS

Se regularizan los tipos de datos de las columnas

Ante la salida de la info del dataset, se observa que la primera columna con tipo de dato que necesita transformación es HH, debido a que se encuentra como objeto y deberia ser númerico.

    - Por ello se analizan los datos de la columna
    - Se transforman a tipo de dato numérico

In [15]:
df_hechos["HH"].unique()

array([4, 1, 7, 0, 5, 18, 19, 15, 11, 22, 16, 9, 23, 6, 10, 17, 12, 8, 20,
       21, 14, 3, 2, 13, 'SD'], dtype=object)

El campo que contiene SD, no se modifica debido a que en la columna HORA donde tambien debería existir dato, se encuentra SD. Por lo que no podemos calcular un reemplazo. Otra opción sería calcular la mediana de las horas y reemplazarlas pero podria no ser representativo en este caso. Por lo que se define mantenerlo en SD

In [16]:
df_hechos["HH"] = pd.to_numeric(df_hechos["HH"], errors="coerce")
df_hechos["HH"] = df_hechos["HH"].astype("Int64")

In [17]:
# Filtrar filas con posiciones no válidas
valid_positions = ~(df_hechos['pos x'].str.contains('[^0-9.,-]') | df_hechos['pos y'].str.contains('[^0-9.,-]') | (df_hechos['pos x'] == '.') | (df_hechos['pos y'] == '.'))

# Establecer posiciones no válidas en NaN
df_hechos.loc[~valid_positions, ['pos x', 'pos y']] = np.nan

# Convertir las cadenas a tipo numérico (float)
df_hechos['pos x'] = df_hechos['pos x'].str.replace(',', '.').astype(float)
df_hechos['pos y'] = df_hechos['pos y'].str.replace(',', '.').astype(float)

In [18]:
df_hechos.head(3)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.",DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.390403,-34.631894,MOTO-AUTO,MOTO,AUTO


La columna XY (CABA) posee coordenadas para poder ubicar en un mapa , las extraemos para poder utilizar

In [19]:
def extraer_coordenadas(texto):
    # Utilizamos una expresión regular para extraer las coordenadas
    coordenadas = re.findall(r'\d+\.\d+', texto)
    if len(coordenadas) == 2:
        return float(coordenadas[0]), float(coordenadas[1])
    else:
        return None, None

# Aplicar la función a la columna 'XY (CABA)'
df_hechos['coordenada_x'], df_hechos['coordenada_y'] = zip(*df_hechos['XY (CABA)'].map(extraer_coordenadas))

# Filtrar los registros que no tienen coordenadas válidas
df_hechos = df_hechos[(df_hechos['coordenada_x'].notnull()) & (df_hechos['coordenada_y'].notnull())]


In [20]:
df_hechos.head(3)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO,coordenada_x,coordenada_y
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO,98896.782384,93532.434378
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS,95832.055711,95505.416420
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.390403,-34.631894,MOTO-AUTO,MOTO,AUTO,106684.290900,99706.576878


In [21]:
#Se observan las columnas resultantes

df_hechos.columns

Index(['ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH',
       'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'Calle', 'Cruce',
       'Dirección Normalizada', 'COMUNA', 'XY (CABA)', 'pos x', 'pos y',
       'PARTICIPANTES', 'VICTIMA', 'ACUSADO', 'coordenada_x', 'coordenada_y'],
      dtype='object')

En la columna 'Comuna' tenemos una numeracion del 1 al 15. Esta numeración identifica a cada una de las divisiones comunales de la ciudad autónoma de Buenos Aires.

Para agregar valor al dataset, agregaremos la descripción de cada una de las comunas, con los Barrios que la conforman.

Para eso utilizaremos el dataset 'comunas' importado junto al dataset homicidios. Este dataset fue descargado del siguiente sitio web: https://buenosaires.gob.ar/comunas

In [25]:
#Se realiza el merge a través de la columna en comun `Comuna` y nos va a quedar anexada la columna `Barrios`

df_hechos=df_hechos.merge(comunas, on="COMUNA", how="left")

#Modifico las mayúsculas por minúsculas
df_hechos = df_hechos.rename(columns={
    "BARRIOS": "Barrios"
})

In [26]:
df_hechos.head(3)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO,coordenada_x,coordenada_y,Barrios
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,8,Point (98896.78238426 93532.43437792),-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO,98896.782384,93532.434378,VILLA LUGANO - VILLA RIACHUELO - VILLA SOLDATI
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,9,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS,95832.055711,95505.416420,LINIERS - MATADEROS - PARQUE AVELLANEDA
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,1,Point (106684.29090040 99706.57687843),-58.390403,-34.631894,MOTO-AUTO,MOTO,AUTO,106684.290900,99706.576878,CONSTITUCION - MONTSERRAT - PUERTO MADERO - RE...


Las colummas tienen nombres en mayúsculas y también en minúsculas. Se estandarizan a modo de buena práctica

In [27]:
#Cambio la primer letra a mayúscula
df_hechos.columns = [x.capitalize() for x in df_hechos.columns]
# Reemplazo guiones por espacios
df_hechos.columns = df_hechos.columns.str.replace("_", " ")
#Renombra columnas 
df_hechos= df_hechos.rename(columns={"N victimas": "Num víctimas",
                                            "Aaaa": "Año",
                                            "Mm":"Mes",
                                            "Dd": "Día",
                                            "Hora": "Hora completa",
                                            "Hh": "Hora"}) 

df_hechos.columns

Index(['Id', 'Num víctimas', 'Fecha', 'Año', 'Mes', 'Día', 'Hora completa',
       'Hora', 'Lugar del hecho', 'Tipo de calle', 'Calle', 'Cruce',
       'Dirección normalizada', 'Comuna', 'Xy (caba)', 'Pos x', 'Pos y',
       'Participantes', 'Victima', 'Acusado', 'Coordenada x', 'Coordenada y',
       'Barrios'],
      dtype='object')

Al analizar los datos de la columna Día se muestra solo el numero del día y no el nombre correspondiente por lo que se crea la columna Día semana donde se corresponde cada fecha con un dia de la semana para poder analizar luego como se relacionan los siniestros con los dias de la semana

In [28]:
df_hechos.Día

0       1
1       2
2       3
3      10
4      21
       ..
677    13
678    20
679    30
680    15
681    18
Name: Día, Length: 682, dtype: int64

In [29]:
# Creo una columna nuevas que se llama 'Día Semana' que contenga el nombre del día de la semana

df_hechos['Día semana'] = df_hechos['Fecha'].dt.strftime('%A')

#Creo un diccionario para modificar a español los nombres de los dás de la semana y lo aplico a la columna nueva
dic={  'Friday':'Viernes',
       'Saturday': 'Sabado',
       'Sunday': 'Domingo',  
       'Thursday': 'Jueves',
        'Monday': 'Lunes',
        'Wednesday': 'Miercoles',
       'Tuesday': 'Martes'}
df_hechos['Día semana'] =df_hechos['Día semana'].replace(dic)

In [30]:
df_hechos.head(3)

,Id,Num víctimas,Fecha,Año,Mes,Día,Hora completa,Hora,Lugar del hecho,Tipo de calle,...,Xy (caba),Pos x,Pos y,Participantes,Victima,Acusado,Coordenada x,Coordenada y,Barrios,Día semana
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,Point (98896.78238426 93532.43437792),-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO,98896.782384,93532.434378,VILLA LUGANO - VILLA RIACHUELO - VILLA SOLDATI,Viernes
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS,95832.055711,95505.416420,LINIERS - MATADEROS - PARQUE AVELLANEDA,Sabado
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,Point (106684.29090040 99706.57687843),-58.390403,-34.631894,MOTO-AUTO,MOTO,AUTO,106684.290900,99706.576878,CONSTITUCION - MONTSERRAT - PUERTO MADERO - RE...,Domingo


La colmuna Comuna se modifica agregando el termino Comuna antes del número, a fin de lograr una variable categórica que me simplifique el análisis

In [31]:
df_hechos['Comuna'].unique()

array([ 8,  9,  1, 11, 15,  4,  7, 12,  3, 13, 14, 10,  6,  2,  5],
      dtype=int64)

In [32]:
#Agrego la palabra Comuna delante del dato numérico
#se aplica la función lambda a cada fila del dataframe 
df_hechos["Comuna"] = df_hechos.apply(lambda x: "Comuna "+str(x["Comuna"]), axis=1)

In [33]:
df_hechos['Comuna'].unique()

array(['Comuna 8', 'Comuna 9', 'Comuna 1', 'Comuna 11', 'Comuna 15',
       'Comuna 4', 'Comuna 7', 'Comuna 12', 'Comuna 3', 'Comuna 13',
       'Comuna 14', 'Comuna 10', 'Comuna 6', 'Comuna 2', 'Comuna 5'],
      dtype=object)

Se realiza un resumen descriptivo de los datos en homicidios-hechos

In [34]:
dfSummary(df_hechos)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,Id[object],1. 2016-00012. 2019-00513. 2019-00434. 2019-00445. 2019-00456. 2019-00467. 2019-00478. 2019-00489. 2019-004910. 2019-005011. other,1 (0.1%)1 (0.1%)1 (0.1%)1 (0.1%)1 (0.1%)1 (0.1%)1 (0.1%)1 (0.1%)1 (0.1%)1 (0.1%)672 (98.5%),"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8g+/7EAAAACXBIWXMAAA9hAAAPYQGoP6dpAAADXElEQVR4nO3cwWkbYRRG0TdGOPZGQhhcQtKDi0ixKUJFpAdjPIvY8WaySBYpINwB5ZwK/sWFh2A+Ldu2DRRu9n4A/w+xkREbGbGRERsZsZERGxmxkREbGbGRuVmW5cuyLHd7P4TrdzMzn2fm094P4fo5o2TERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRG5mZmvs/Mz70fwvVb/IsRFWeUjNjIWFeRsa4i44ySERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZExeCFj8ELGGSUjNjIGL2QMXsg4o2TERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZAxeyBi8kHFGyYiNjNjIWFeRsa4i44ySERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZGxriJjXUXGGSUjNjIGL2QMXsg4o2TERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZAxeyBi8kHFGyYiNjMELGYMXMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZGxriJjXUXGGSUjNjLWVWSsq8g4o2TERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZAxeyBi8kHFGyYiNjMELGYMXMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZgxcyBi9knFEyYiMjNjLWVWSsq8g4o2TERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZKyryFhXkXFGyYiNjMELGYMXMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZgxcyBi9knFEyy8zcz8zt3g/hqn1s2/Z2eHh4+Ho8Hs97v4brta7ry7Is3w7H4/H89PT0djqd3vd+FNfn9fX17nK5nJ+fn28PMzOn0+n98fHxx94P42rdz/iBQEhsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERuYw8/szkL0fwnX6u63Duq4vl8vlPH8+A4F/bV3Xl5n58Fk4hY9t296sq8j4gUBGbGTERkZsZMRGRmxkxEZGbGTERuYXwgdpOf758ysAAAAASUVORK5CYII="">",0(0.0%)
2,Num víctimas[int64],Mean (sd) : 1.0 (0.2)min < med < max:1.0 < 1.0 < 3.0IQR (CV) : 0.0 (5.8),3 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8g+/7EAAAACXBIWXMAAA9hAAAPYQGoP6dpAAABs0lEQVR4nO3XQWrCUBCH8ZkiqQ/khZCaY3gAD+FhPYRbTxLkZdGoVF53hVJaTbToX77fNplhFh8RPedswKN7ufcBwCUm515w92Bmxcj9x5xzP3IW+PJnqO4e6rpexRirMcu7rtu5+5pYca1zX9Qixlgtl8u+LMv9kMUppelms6nati3MjFBxlbM//WZmZVnum6Z5H7E/jJgBfuDPFCQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiRM7n3AM3L3YGbFyPFjzrm/5T3PgFBvzN1DXderGGM1Zr7rup27r4n1u4tCTSlNhy5OKU1Pp9PEzKK7Dz5MWJzNZvPFYvERQjgMGez7/nW73c7btn1z9+6/DnxQh5zz/reHnwblSFGwSZ17AAAAAElFTkSuQmCC"">",0(0.0%)
3,Fecha[datetime64[ns]],"Min: 2016-01-01Max: 2021-12-30Duration: 2,190 days",589 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAABNCAYAAACxBha+AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8g+/7EAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACkUlEQVR4nO3dTYriQByG8X+NrUSFiAi68wIDs3DpIeawc4RZeAJv4E4QiZjpYIyZzcAsuv1ItXnNx/PblwT6KVNVaYnL89wAhW+vvgC0B7FB5q2MD3XO9c2s5zH0lOf5+7OvB9Xw9Nicc/3JZPIzDMNx0bGHw2HvnPulCu4Lk8KMiVFYGd9svTAMx8vl8n00GiWPDoqiKFitVuPdbtczs9L/iF+ZFGb6idEEpdxGzcxGo1EynU7/FBzWL+ViPuc1Kcz0E6MpSoutLjwnhZl2YjQCu1HIEBtkiA0yxAaZRmwQPM/Lwsvl0i3jevC52sfme16WpmmQZdn3JEl+m5nPbhQF1T428zwv22w24/V6PTifz50yLw7/NSE2Myt+Xrbf71txTlal59SVii3Lsq6Zhc65IsNYe11RtefUN2NTLrzjOO6mafpjPp93Op3Ow7dD1l43Veo59dXY1Avv0+n0FgTBYLFYJLPZbP/oONZe91XlOfWtb7aXLLyHwyFrr4a6u2Zj4Y1n4QkCZIgNMsQGmUqds9WJ55mgWYt/u0BsHnzPBM3a/dsFYvPgeybY9t8uENsXFD0T/Ke1R0NsECDDN5tYmzcWxCbU9o0FsQm1fWNBbC9Qh41FGbd7YsMHZd3uiQ0flHW7JzZc9ezbPedskCE2yBAbZIgNMsQGGWKDDLFBhtggQ2yQITbIEBtkiA0yxAYZYoMMsUGG2CBDbJAhNsgQG2SIDTLEBhl+XVUTTXghCbHVQFNeSEJsNdCUF5IQW43U/YUkbBAgQ2yQITbIEBtkiA0yxAYZYoMMsUGG2CBDbJAhNsgQG2SIDTLEBhlig8zd/2eLoigo8oHH4zEwM4vjONhut4OmjavTtarH3WvlL6rovNJurq/hAAAAAElFTkSuQmCC"">",0(0.0%)
4,Año[int64],Mean (sd) : 2018.2 (1.7)min < med < max:2016.0 < 2018.0 < 2021.0IQR (CV) : 3.0 (1201.6),6 distinct values,"<img src = ""data:image/png;ba